# DeepBind integration in Motif-Raptor

### Description


In this notebook we show how to integrate the binding scores of DeepBind (Alipanahi et. Al Nature Biotech 2015: https://www.nature.com/articles/nbt.3300) in Motif-Raptor. We will focus on the step 2 of the pipeline i.e. the ranking of potential causal TFs for each of the prioritized cell types obtained in step 1 based on a set of SNPs in cell type specific DHS.


### Requirements

A working installation of Motif-Raptor (see instructions here: https://github.com/pinellolab/MotifRaptor#installation)

A working installation of DeepBind (see instructions here: http://tools.genes.toronto.edu/deepbind/)


### 0. Introduction

In this example we will use one DeepBind model for NFKB1 (https://github.com/pinellolab/MotifRaptor#installation) using cell type specific chromatin accessible sites in CD8+ cells and summary statistics for rheumatoid arthritis (Okada et al., 2014). However this preocedure is general and can be extended to all the precomputed models available in DeepBind and other cell type and GWAS studies.

The input to this integration is the output from the step 2 from Motif-Raptor and in particular the following files (see the tutorial for more information on how to run step 1 and step 2: https://github.com/pinellolab/MotifRaptor#tutorial ):

(1)"hit_snps_seq_pickle_celltypeid.df.txt":

    This file is from Motif-Raptor step 2 (step2_out). It contains SNP hits defined by the summary statistics and overlapping the DHS of the target cell type. From this file, you can easily get the flanking sequences around each SNP for reference and alternative alleles.
    
    An example of this file will be used in the step 1 of this notebook.

(2) "motifid.scores":

    This file contains background SNP IDs from Motif-Raptor step 2 (step2_out/motif_result/background_files). It's handy to use this file to generate the background data set. However, you can also define your own background data set as long as it follows the required format as input to Deepbind.
    
    An example of this file will be used in the step 3 of this notebook.

### 1. Get list of SNPs from Motif-Raptor

In [1]:
import numpy as np
import pandas as pd
from scipy.stats import norm

This file is for Rheumatoid Arthritis in CD8+ cells, which can be obtained from Motif-Raptor step2 output. It's described in the above introduction section.

In [2]:
filename="RA/step2_out/hit_snps_seq_pickle_ENCFF512IML.df.txt"
hit_snp_pd=pd.read_csv(filename,sep="\t",header=0)
hit_snp_pd.head(3)

,ID,CHR,POS,REF,ALT,seq_ref,seq_alt,pos_start_ref,pos_end_ref,pos_start_alt,pos_end_alt
0,rs11585048,1,2534087,T,C,ACCAGGTCACGTCCTGTCATACCCCCACTCTTCTCGGGCAGAGTCT...,ACCAGGTCACGTCCTGTCATACCCCCACTCCTCTCGGGCAGAGTCT...,30,30,30,30
1,rs10797435,1,2534801,G,C,GCAGGGTGAGAGTCGGCTGAGGGAGTGGGGGACTTGCCCCTTCCCA...,GCAGGGTGAGAGTCGGCTGAGGGAGTGGGGCACTTGCCCCTTCCCA...,30,30,30,30
2,rs4648652,1,2535758,A,G,AAAGGACGCCGGGCAGAGCTGGGTGGCCTCAGCCACCATGGCCCCC...,AAAGGACGCCGGGCAGAGCTGGGTGGCCTCGGCCACCATGGCCCCC...,30,30,30,30


We collect flanking sequences for reference and alternative allels around each SNP, and then we write these sequences to a sequence file *.seq, which is required for deepbind program.
In the "demo_RA_CD8.seq" generated below, we use consecutive two lines to store the sequences (reference and alternative) for each SNP.

In [3]:
seq_list=hit_snp_pd['seq_ref'].tolist()+hit_snp_pd['seq_alt'].tolist()
outfile=open("demo_RA_CD8.seq","w")
for seq in seq_list:
    outfile.write(seq+"\n")
outfile.close()

### 2. Run a deepbind model

In [4]:
deepbind_id="D00540.002" # This is NFKB1 model from deepbind

We need to write deepbind_id (each ID in one line) into a file "demo_input.ids"

In [5]:
!touch demo_input.ids
!echo "D00540.002" > demo_input.ids

Using the obtained file ("demo_RA_CD8.seq"), you can run deepbind using the following command:

In [6]:
!./deepbind demo_input.ids < demo_RA_CD8.seq > demo_deepbind_out_RA_CD8.txt

### 3. Build the null model for a deepbind model

This is similar to the previous step:
You need to organize the background SNP using the same format as in "demo_RA_CD8.seq"

You can also define your own background SNP set based on the similar format as following.

In [7]:
backgroundfilename="RA/step2_out/motif_result/background_files/MA0105.1__NFKB1.scores"
background_df=pd.read_csv(backgroundfilename,sep="\t",header=0)
background_df[['Genome','pos_SNP','SNP_in_ref','SNP_in_alt','ID']].head(3)

,Genome,pos_SNP,SNP_in_ref,SNP_in_alt,ID
0,chr1,13118,A,G,rs200579949
1,chr1,540426,T,C,rs540135598
2,chr1,568709,A,G,rs148329687


Get the flanking sequences using "twobitreader". If you have installed Motif-Raptor, this tool will be already available.

You may also use the command line version from UCSC: https://genome.ucsc.edu/goldenPath/help/twoBit.html

In [8]:
import twobitreader
#change the Database path depending on where you unpack the Database.zip from Motif-Raptor
genome = twobitreader.TwoBitFile("Database/hg19/hg19.2bit")

def getseq_from_genome(from_genome,chromosomenum, start_pos, end_pos):
    chromosome = from_genome[chromosomenum]
    seq = chromosome.get_slice(start_pos, end_pos)
    return seq.upper()

#we only demo a few SNPs from the background, you may change this number
testing_num=500
testing_df=background_df[['Genome','pos_SNP','SNP_in_ref','SNP_in_alt','ID']].head(testing_num)

window_size=30
outfile=open("demo_RA_CD8_background.seq","w")
for index in testing_df.index:
    chrom=testing_df.loc[index]['Genome']
    pos=int(testing_df.loc[index]['pos_SNP'])
    start_pos=pos-window_size
    end_pos=pos+window_size
    ref=str(testing_df.loc[index]['SNP_in_ref']).upper()
    alt=str(testing_df.loc[index]['SNP_in_alt']).upper()
    seq=getseq_from_genome(genome,chrom, start_pos, end_pos)
    ref_seq=seq[0:window_size]+ref+seq[window_size+1:2*window_size+1]
    alt_seq=seq[0:window_size]+alt+seq[window_size+1:2*window_size+1]
    outfile.write(ref_seq+"\n")
    outfile.write(ref_seq+"\n")
outfile.close()



Using the obtained file ("demo_RA_CD8_background.seq"), you can run deepbind using the following command:

In [9]:
!./deepbind demo_input.ids < demo_RA_CD8_background.seq > demo_deepbind_out_RA_CD8_background.txt

### 4. Create motif modulation score for deepbind

This function is used to calculate binding scores, disruption scores, and modulation scores for SNPs in "filename"

[1] filename: deepbind output

[2] deepbind_id: deepbind model ID

[3] bind_scaler,dirsupt_scaler_pos,disrupt_scaler_neg: these are scalers used to normalize the score. If not set, it will calcualte these scalers on the current data.

In [10]:
def get_SNP_scores(filename, deepbind_id, bind_scaler=0,dirsupt_scaler_pos=0,dirsupt_scaler_neg=0):
    bind_scores=list()
    disrupt_scores=list()
    deepbind_df=pd.read_csv(filename,sep="\t",header=0)
    deepbind_column=deepbind_df[deepbind_id].tolist()
    n=int(len(deepbind_column)/2)
    
    for i in range(0,n):
        a=deepbind_column[i]
        b=deepbind_column[i+1]
        bind_score = max(a,b)
        if bind_score>=0:
            bind_scores.append(bind_score)
            disrupt_score=b-a
            disrupt_scores.append(disrupt_score)
    if bind_scaler==0:
        bind_scaler=max(bind_scores)
        dirsupt_scaler_pos=max([i for i in disrupt_scores if i >= 0] or None)
        dirsupt_scaler_neg=min([i for i in disrupt_scores if i < 0] or None)
    scaled_bind_scores=list(np.divide(np.array(bind_scores),bind_scaler))
    scaled_disrupt_scores=list()
    for score in disrupt_scores:
        if score>=0:
            newscore=score/dirsupt_scaler_pos
        else:
            newscore=-score/dirsupt_scaler_neg
        scaled_disrupt_scores.append(newscore)
    scaled_mod_scores=list(np.multiply(scaled_bind_scores,scaled_disrupt_scores))
    return bind_scaler,dirsupt_scaler_pos,dirsupt_scaler_neg,scaled_bind_scores,scaled_disrupt_scores,scaled_mod_scores

Calculate scores for target and background SNPs

In [11]:
target_filename="demo_deepbind_out_RA_CD8.txt"
background_filename="demo_deepbind_out_RA_CD8_background.txt"

In [12]:
bind_scaler,dirsupt_scaler_pos,dirsupt_scaler_neg,scaled_bind_scores_bg,scaled_disrupt_scores_bg,scaled_mod_scores_bg \
    = get_SNP_scores(background_filename,deepbind_id)

In [13]:
bind_scaler,dirsupt_scaler_pos,dirsupt_scaler_neg,scaled_bind_scores_target,scaled_disrupt_scores_target,scaled_mod_scores_target \
    = get_SNP_scores(target_filename,deepbind_id,bind_scaler,dirsupt_scaler_pos,dirsupt_scaler_neg)

### 5. Assess the significance

Using the target and background modulation scores (i.e. "scaled_mod_scores_target" and "scaled_mod_scores_bg"), you can calculate p-values for potential disruption, enhancement and dual events for TF binding.

In [14]:
target_data=scaled_mod_scores_target
bg_data=scaled_mod_scores_bg

pop_mean=np.mean(bg_data)
pop_var=np.var(bg_data)
test_mean=pop_mean

test_var=pop_var/(len(target_data))


t=(np.mean(target_data)-test_mean)/((test_var)**(0.5))
report_pvalue_left=norm.cdf(t)
report_pvalue_right=1-report_pvalue_left
#print(str(test_mean))
#print(str(test_var)) 
abs_target_data=np.abs(target_data)
abs_bg_data=np.abs(bg_data)
abs_pop_mean=np.mean(abs_bg_data)
abs_pop_var=np.var(abs_bg_data)
abs_test_mean=abs_pop_mean
abs_test_var=abs_pop_var/(len(abs_target_data))
if abs_test_var==0:
    abs_test_var=1E-20
abs_t=(np.mean(abs_target_data)-abs_test_mean)/((abs_test_var)**(0.5))
report_pvalue_abs=1-norm.cdf(abs_t)
print("Disruption p-value: "+str(report_pvalue_left))
print("Enhancement p-value: "+str(report_pvalue_right))
print("Dual p-value: "+str(report_pvalue_abs))



Disruption p-value: 0.26782865836842923
Enhancement p-value: 0.7321713416315707
Dual p-value: 0.03529465255820752
